In [ ]:
from twython import Twython
import json 
import csv


credentials = {}
credentials["API_KEY"] = "kfMHh8BuUXMCQ9S57eHlZPtp3"
credentials["API_SECRET"] = "YBBa1e6czkDK30KOpdMPN20QCWao2M8ONvCodfnH8gWW1AOXIu"
credentials["ACCESS_TOKEN"] = "1501925600105218050-lOCbvbbF4Ikoy9ublZQi4NAiGsAN84"
credentials["ACCESS_TOKEN_SECRET"] = "cS5qbsBa9IvF6ORoKs6UZtGz5KbLe0nY9lINwWGeK35ao"


with open("credentials.json", "w") as file:
    json.dump(credentials, file)








In [ ]:
results = twitter.search(q="Jungkook")
r = results["statuses"][0]

with open("sample.json", "w") as file:
    json.dump(r,file)

results

In [3]:
from twython import Twython
import json

credentials = {}
with open("credentials.json", "r") as file:
    credentials = json.load(file)

twitter = Twython(app_key=credentials["API_KEY"],
                 app_secret=credentials["API_SECRET"], 
                 oauth_token=credentials["ACCESS_TOKEN"],
                 oauth_token_secret=credentials["ACCESS_TOKEN_SECRET"])


class Twitter_Utility:

    def __init__(self):
        self.results = {}

    def process_tweet(self, data):
        tweet = {}
        tweet["location"] = data["user"]["location"]
        tweet["text"] = data["text"]
        tweet["client_platform"] = data["source"]
        tweet["hashtags"] = [hashtag["text"] for hashtag in data["entities"]["hashtags"]]
        return tweet


    def csv_set_header(self, tweet):
         with open("saved_tweets.csv", "a", encoding="utf-8",newline="") as file:
            header = [key for key in self.process_tweet(tweet)]
            csv_writer = csv.writer(file)
            csv_writer.writerow(header)
        
    def save_to_csv(self, tweet):
        with open("saved_tweets.csv", "a", encoding="utf-8", newline="") as file:
            csv_writer = csv.writer(file)
            tweet_content = [tweet[key] for key in tweet]
            csv_writer.writerow(tweet_content)


    def search_tweet(self, search_query,count):
        results = twitter.search(q=search_query, count= count)   
        self.results = results  
        
    
    def save_tweets(self):
        if self.results.get("statuses"):
                self.csv_set_header(self.results["statuses"][0])
                for result in self.results["statuses"]:
                    tweet = self.process_tweet(result) 
                    self.save_to_csv(tweet)




twitter_util = Twitter_Utility()
twitter_util.search_tweet("Ukraine", count=100)
twitter_util.save_tweets()

NameError: name 'csv' is not defined